In [1]:
# 8-1
!mkdir chap8
%cd ./chap8

/Users/maton/BERT_Practice/chap8


/Users/maton/.pyenv/versions/3.10.13/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# 8-3
import itertools
import random
import json
from tqdm import tqdm
import numpy as np
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl

# 日本語学習済みモデル
MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"

In [3]:
# 8-4（説明コメント付き：Unicode 正規化 NFKC による表記ゆれの吸収）
# -----------------------------------------------------------------------------
# 目的：
#  - Unicode の **互換性分解 + 合成**（NFKC）で、全角/半角・互換文字（例：半角ｶﾅ・ローマ数字・① 等）
#    を正規化し、検索/重複排除/機械学習前処理での表記ゆれを減らす。
#
# 理論メモ（NFC/NFD/NFKC/NFKD の違い）：
#  - NFC: 正規分解 → 合成（互換性は無視）。見た目を保ちながら正規の合成形式へ。
#  - NFD: 正規分解のみ（結合文字にバラす）。検索・照合の下処理に使うことがある。
#  - NFKC: **互換分解**（見た目は同じでも “意味的に同一視される” 文字を分解）→ 合成。
#          例：全角英数/記号、半角ｶﾅ、ローマ数字 Ⅳ、丸数字 ① などを通常の文字に畳み込む。
#  - NFKD: 互換分解のみ。
# 互換分解は “情報の落ち” が起こり得る（例：①→1、㍍→メートル→m など）。監査用途では注意。
# -----------------------------------------------------------------------------

import unicodedata

# NFKC で正規化する関数
# - 全角英数→半角、半角ｶﾅ→全角カタカナ、結合記号の統合 等を一括で行う
normalize = lambda s: unicodedata.normalize("NFKC", s)

# 動作例：全角/半角の統一（学習前の前処理・検索キー作成などで有効）
print(f'ＡＢＣ -> {normalize("ＡＢＣ")}')  # 全角アルファベット → 半角 "ABC"
print(f'ABC -> {normalize("ABC")}')  # 既に半角 → 変化なし
print(f'１２３ -> {normalize("１２３")}')  # 全角数字 → 半角 "123"
print(f'123 -> {normalize("123")}')  # 既に半角 → 変化なし
print(f'アイウ -> {normalize("アイウ")}')  # 全角カタカナ → 変化なし
print(f'ｱｲｳ -> {normalize("ｱｲｳ")}')  # 半角ｶﾅ → 全角カタカナ "アイウ"

# -----------------------------------------------------------------------------
# 追加の知見（必要ならテストして確認）：
#  - 濁点付き半角ｶﾅ（例：ｶﾞ）も結合が解決され全角「ガ」に統一される：
#      normalize("ｶﾞ") == "ガ"
#  - 互換文字の折り畳み：
#      normalize("ⅠⅡⅢ") -> "III"  （ローマ数字 → ラテン大文字）
#      normalize("①②")   -> "12"   （丸数字 → 通常数字）
#  - 正規化は “可逆でない” ことがあるため、**原文は別項目で保存**しておくと安全。
# -----------------------------------------------------------------------------

ＡＢＣ -> ABC
ABC -> ABC
１２３ -> 123
123 -> 123
アイウ -> アイウ
ｱｲｳ -> アイウ


In [4]:
# 8-5（説明コメント付き：日本語BERT用 NER 前処理ユーティリティ）
# =============================================================================
# 前提：
#  - BertJapaneseTokenizer（transformers）を継承して、固有表現抽出（NER）向けの
#    2種類のエンコード（教師あり/教師なし）と、モデル出力から固有表現スパンを復元する
#    補助関数を提供するクラスである。
#  - ラベリング方式は BIO ではなく「整数 ID によるスパン連結」方式（O=0, その他=タイプID）。
#    → 連続する同一ラベルのトークン列を 1 つのエンティティにまとめる前提。
#  - 重複・ネスト・オーバーラップするエンティティは表現できない（単純スパンのみ）。
#  - prepare_for_model により [CLS]/[SEP] の特殊トークンが付与される前提で、ラベルを先頭/末尾0に整合。
#  - max_length を超える部分は切り捨てられるため、切断されたエンティティは欠落し得る点に注意。
#  - encode_plus_untagged では、MeCab 分かち書き（word_tokenizer）→ WordPiece（subword_tokenizer）
#    で二段階トークナイズを行い、トークン表層を原文へシーケンシャルマッチしてスパンを得る。
#    同一トークンの繰り返しが多い文でも、左から貪欲にマッチしていくため順次対応できる。
#  - return_tensors='pt' 指定時、encoding のみ Tensor 化する（spans はリストのまま返る）。
#    下流でテンソルを期待する場合は適宜変換すること。
# =============================================================================


class NER_tokenizer(BertJapaneseTokenizer):

    def encode_plus_tagged(self, text, entities, max_length):
        """
        文章とそれに含まれる固有表現が与えられた時に、
        符号化とラベル列の作成を行う。

        引数:
          text: str
            元の文章（原文）。エンティティ抽出の対象。
          entities: List[Dict]
            [{'span': [start, end], 'type_id': int}, ...] の形を想定。
            span は原文 text における [start, end) の半開区間、type_id は 0 以外の整数。
          max_length: int
            BERT 入力の最大系列長。prepare_for_model で [CLS]/[SEP] を含む長さに正規化される。

        戻り値:
          encoding: Dict[str, List[int]]
            input_ids / attention_mask / token_type_ids / labels を含む辞書。
            labels は [0]=CLS, 末尾 [0]=SEP/必要に応じ PAD を 0 で詰める。
        """
        # --- 前処理: エンティティを開始位置で昇順ソート ---
        entities = sorted(entities, key=lambda x: x["span"][0])

        # --- 原文を「非固有表現片(O=0)」「固有表現片(=type_id)」に分割して並べる ---
        splitted = []  # 分割後の文字列片を順に蓄積
        position = 0  # 直前に処理した末尾の次インデックス
        for entity in entities:
            start = entity["span"][0]
            end = entity["span"][1]
            label = entity["type_id"]
            # 非固有表現部分（直近の position からエンティティ開始まで）→ ラベル 0
            splitted.append({"text": text[position:start], "label": 0})
            # 固有表現部分（[start:end)）→ ラベル type_id
            splitted.append({"text": text[start:end], "label": label})
            position = end
        # 最後のエンティティ以降の末尾テキストも非固有表現として追加
        splitted.append({"text": text[position:], "label": 0})
        # 空文字は除去（連続するエンティティで start==end 等により空が生じ得るため）
        splitted = [s for s in splitted if s["text"]]

        # --- 片ごとにトークン化し、各トークンへ片に対応するラベルを付与 ---
        tokens = []  # WordPiece トークン列
        labels = []  # トークンに対するラベル（0 or type_id）
        for text_splitted in splitted:
            text = text_splitted["text"]
            label = text_splitted["label"]
            # BertJapaneseTokenizer.tokenize は日本語前処理（MeCab）+ WordPiece を内部で実施
            tokens_splitted = self.tokenize(text)
            labels_splitted = [label] * len(tokens_splitted)
            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        # --- トークン列を ID 列に変換し、BERT 入力辞書へ整形 ---
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids, max_length=max_length, padding="max_length", truncation=True
        )  # prepare_for_model が [CLS]/[SEP] 等を付与し、長さを揃える

        # --- ラベルの特殊トークン・PAD 整合 ---
        # 先頭に CLS=0 を付与、本文ラベルは最大長-2（CLS/SEP）までに切詰め、末尾に SEP=0
        labels = [0] + labels[: max_length - 2] + [0]
        # さらに不足分（PAD 部分）を 0 で埋め、最終的に長さを max_length に一致させる
        labels = labels + [0] * (max_length - len(labels))
        encoding["labels"] = labels

        return encoding

    def encode_plus_untagged(self, text, max_length=None, return_tensors=None):
        """
        文章をトークン化し、それぞれのトークンの文章中の位置も特定しておく。

        引数:
          text: str
            元の文章。
          max_length: Optional[int]
            指定時は padding='max_length', truncation=True で固定長化。
            未指定時は可変長（特殊トークン付与は維持）。
          return_tensors: Optional[str]
            'pt' を指定すると encoding の各値を torch.Tensor(batch次元つき) に変換。

        戻り値:
          encoding: Dict[str, List[int]] or Dict[str, torch.Tensor]
            prepare_for_model の出力（[CLS]/[SEP] 付与後）。
          spans: List[List[int]]
            各トークンに対応する原文上の [start, end) 位置。
            特殊トークン([CLS],[SEP],[PAD])の位置は [-1,-1] のダミーにする。
        """
        # --- 形態素（MeCab）→ サブワード（WordPiece）の二段トークナイズ ---
        tokens = []  # WordPiece トークン
        tokens_original = (
            []
        )  # スパン計算用の「表層文字列」列（'##' 除去/UNK は単語そのまま）
        words = self.word_tokenizer.tokenize(text)  # MeCab による単語列
        for word in words:
            # 単語を WordPiece に分割
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)
            if (
                tokens_word[0] == "[UNK]"
            ):  # 未知語は WordPiece に分割できない → 原単語をそのまま使う
                tokens_original.append(word)
            else:
                # '##' 接頭辞を削って表層形を復元（スパン同定に用いる）
                tokens_original.extend(
                    [token.replace("##", "") for token in tokens_word]
                )

        # --- 原文上のスパンを左から順次マッチで同定（空白/記号も考慮して進める）---
        position = 0  # 原文上の走査位置（左から前進のみ）
        spans = []  # 各トークンの [start, end) を蓄積
        for token in tokens_original:
            l = len(token)
            while 1:
                # 現在位置から長さ l を切り出して一致判定（空白等を飛ばすため不一致なら1文字進める）
                if token != text[position : position + l]:
                    position += 1
                else:
                    spans.append([position, position + l])
                    position += l
                    break
        # ここでの手法は「逐次マッチ」のため、同一部分文字列の繰り返しがあっても左から順に整合が取れる。
        # ただし、原文編集（正規化や空白折り畳み）を別途行う場合は、同じ変換を token 側にも適用しておくこと。

        # --- BERT 入力辞書へ整形（固定長/可変長を分岐）---
        input_ids = self.convert_tokens_to_ids(tokens)
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding="max_length" if max_length else False,
            truncation=True if max_length else False,
        )
        sequence_length = len(
            encoding["input_ids"]
        )  # [CLS]/[SEP] を含む最終トークン列長

        # --- 特殊トークン分のダミー span を付与し、長さを encoding に一致させる ---
        # 先頭 [CLS] 用のダミー
        spans = [[-1, -1]] + spans[: sequence_length - 2]
        # 末尾 [SEP] と PAD 分をダミーで埋める（長さを sequence_length に揃える）
        spans = spans + [[-1, -1]] * (sequence_length - len(spans))

        # --- 必要なら Tensor 化（encoding のみ。spans はリストで返す） ---
        if return_tensors == "pt":
            encoding = {k: torch.tensor([v]) for k, v in encoding.items()}

        return encoding, spans

    def convert_bert_output_to_entities(self, text, labels, spans):
        """
        文章、ラベル列の予測値、各トークンの位置から固有表現を得る。

        引数:
          text: str
            原文。
          labels: Sequence[int]
            各トークンのラベル（O=0, その他は type_id）。[CLS]/[SEP]/[PAD] 位置の分も含む想定。
          spans: Sequence[List[int]]
            各トークンの [start, end)。特殊トークンは [-1,-1]。

        戻り値:
          entities: List[Dict]
            {"name": 原文片, "span": [start,end], "type_id": ラベルID} の配列。
        """
        # --- 特殊トークン（span=-1）に対応するラベル/スパンを除去して本文トークンに限定 ---
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # --- 連続する同一ラベルをまとめて 1 スパンのエンティティに復元 ---
        # itertools.groupby により、labels の連続区間ごとにグルーピングする。
        # 例: labels=[0,0,2,2,0,3] → (0区間)(2区間)(0区間)(3区間)
        # ラベル0（O）は無視し、非0の区間のみエンティティを生成する。
        entities = []
        for label, group in itertools.groupby(enumerate(labels), key=lambda x: x[1]):

            group = list(group)  # group は [(idx,label), ...] の列
            start = spans[group[0][0]][0]  # 区間先頭トークンの start
            end = spans[group[-1][0]][1]  # 区間末尾トークンの end（半開区間の終端）

            if label != 0:  # O 以外のラベルのみエンティティ化
                entity = {
                    "name": text[start:end],  # 原文断片（復元テキスト）
                    "span": [start, end],  # 原文上の [start,end)
                    "type_id": label,  # ラベル ID（BIO ではなく type_id そのもの）
                }
                entities.append(entity)

        # 注意：
        #  - BIO 方式ではないため、同一ラベルが連続すれば 1 エンティティに連結される。
        #    同じ type_id が離れて複数回出現する場合は、それぞれ別エンティティとして復元される。
        #  - ネスト/オーバーラップは扱えない（単純連続区間のみ）。
        #  - max_length による切詰めでエンティティの一部が失われると、復元されない場合がある。

        return entities

In [5]:
# 8-6
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [6]:
# 8-7（説明コメント付き：NER_tokenizer.encode_plus_tagged の動作確認）
# =============================================================================
# 目的：
#  - 原文 `text` と、原文上のスパン（[start, end) の半開区間）＋ラベルIDからなる `entities`
#    を与え、BERT 入力（input_ids/attention_mask/token_type_ids）に整形すると同時に、
#    各トークンに対応する **ラベル列**（O=0, エンティティは type_id）を作成する。
#
# 理論メモ：
#  - encode_plus_tagged は「BIO 方式」ではなく **連結方式**（同一ラベルが連続するトークン列を1つの
#    エンティティとみなす）前提でラベルを作る。特殊トークン [CLS]/[SEP]/[PAD] は 0 として埋める。
#  - `span=[start,end]` は **半開区間**（start 文字を含み end 文字を含まない）。インデックスは 0 始まり。
#  - トークン化は WordPiece なので、1語が複数トークンに分割されても、分割前の片に付けたラベルが
#    そのまま各トークンに複製される設計。
#  - `max_length` 超過分は切り捨てられるため、末尾側のエンティティが途中で切れると、対応トークンに
#    ラベルが乗らず欠落し得る点に注意（長さ設計 or スライディングで対処）。
#  - 直前に正規化（例：NFKC）を行った場合、**スパンは正規化後の文字列基準**で与えること（前後で
#    文字長が変わるとズレる）。
# =============================================================================

# （互換/保守）NER_tokenizer が未インポート/未定義でも動くように最小限のフォールバックを用意
try:
    NER_tokenizer
except NameError:
    # ここではクラス本体は 8-5 で定義済み想定。未定義なら明示エラーにする。
    raise RuntimeError(
        "NER_tokenizer クラスが未定義です。先に 8-5 の定義を実行してください。"
    )

# 既存の `tokenizer` が NER_tokenizer でない場合は、同名で置き換えても構わない運用なら差し替える
try:
    tokenizer
    has_encode_tagged = hasattr(tokenizer, "encode_plus_tagged")
except NameError:
    has_encode_tagged = False

if not has_encode_tagged:
    # 学習時と同じ語彙を使う（未定義なら東北大BERTにフォールバック）
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "tohoku-nlp/bert-base-japanese-whole-word-masking"
    tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# --------------------------- 元のコード（＋説明コメント） ---------------------------

# 入力テキスト（インデックスは 0 始まり）
# 文字位置: 0:昨,1:日,2:の,3:み,4:ら,5:い,6:事,7:務,8:所,9:と,10:の,11:打,12:ち,13:合,14:わ,15:せ,16:は,17:順,18:調,19:だ,20:っ,21:た,22:。
text = "昨日のみらい事務所との打ち合わせは順調だった。"

# エンティティの指定：
#  - name は人間可読の補助で、実際のラベル付与は span と type_id によって行われる
#  - span=[3,9) → 原文中の「みらい事務所」（3〜8文字目）を指定（end=9 は「と」なので非包含）
#  - type_id=1 → O（背景）を 0 とするクラス設計の「1番」カテゴリ
entities = [{"name": "みらい事務所", "span": [3, 9], "type_id": 1}]

# ラベル列作成つきの符号化（最大長は [CLS]/[SEP] を含むトークン長に正規化される）
# labels は先頭[CLS]=0、本文（max_length-2 まで）、末尾[SEP]=0、残りPAD=0 で埋められる。
encoding = tokenizer.encode_plus_tagged(text, entities, max_length=20)

# 出力の中身は Hugging Face の標準キー（input_ids, attention_mask, token_type_ids）に加え、
# 1:1 に対応する labels が入っている。確認用にそのまま表示。
print(encoding)

# 追加の確認（任意）：
# - トークン列長とラベル列長が一致していること
# - ラベル中の 1 の連続区間が「みらい事務所」の WordPiece 分割数に一致していること
# 例：
# tokens = tokenizer.convert_ids_to_tokens(encoding["input_ids"])
# print(tokens)
# print(encoding["labels"])

{'input_ids': [2, 10271, 28486, 5, 546, 3000, 1518, 233, 13, 5, 1878, 2682, 9, 10750, 308, 10, 8, 3, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], 'labels': [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [7]:
# 8-8
text = "騰訊の英語名はTencent Holdings Ltdである。"
encoding, spans = tokenizer.encode_plus_untagged(text, return_tensors="pt")
print("# encoding")
print(encoding)
print("# spans")
print(spans)

# encoding
{'input_ids': tensor([[    2,     1,     5,  1543,   125,     9,  6749, 28550,  2953, 28550,
         28566, 21202, 28683, 14050, 12475,    12,    31,     8,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
# spans
[[-1, -1], [0, 2], [2, 3], [3, 5], [5, 6], [6, 7], [7, 9], [9, 10], [10, 12], [12, 13], [13, 14], [15, 18], [18, 19], [19, 23], [24, 27], [27, 28], [28, 30], [30, 31], [-1, -1]]


In [8]:
# 8-9
labels_predicted = [0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
entities = tokenizer.convert_bert_output_to_entities(text, labels_predicted, spans)
print(entities)

[{'name': '騰訊の', 'span': [0, 3], 'type_id': 1}, {'name': 'ncent Holdings Ltdで', 'span': [9, 28], 'type_id': 1}]
